In [1]:
cd ..

/home/jovyan/dsi_plus/Projects/Project-4-semantic_search


In [2]:
%run lib/__init__.py

# Project 4 - Semantic Search

## Develop extraction process for data from Wikipedia

### Goal: Extract data from specified categories ,  code should be generic so can be pulled for any category on Wikipedia.  Final solution should be a single function in a py file will that can be run by any user.

Target categories
- https://en.wikipedia.org/wiki/Category:Machine_learning (explore the category structure)
- https://www.mediawiki.org/wiki/API:Main_page (start reading the API documentation)

For requested category pull...
    - All unique subcategories to depth of 2( default setting, can be changed), store in category_collection
    - All unique pages for unique subcategories 
    - For each page retrieve Article extract plus list of categories page belongs to, store in page_collection
  

In [3]:
#For development process use small category within Machine Learning 
cat_name='Classification algorithms'

In [4]:
wiki=WikiAPI('Category:'+cat_name, run=False)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Extract all unique subcategories from main category
( add list to Mongo category_collection )

In [5]:
%%time
subcats, subcat_ids =wiki.wiki_cats()

CPU times: user 150 ms, sys: 0 ns, total: 150 ms
Wall time: 1.68 s


In [6]:
len(subcats) ,len(subcat_ids) 

(5, 5)

In [7]:
subcats_df=pd.DataFrame(wiki.subcats)
subcats_df.head(3)

,ns,pageid,title
0,14,42936114,Category:Artificial neural networks
1,14,17594154,Category:Decision trees
2,14,3985352,Category:Ensemble learning


In [8]:
wiki.write_subcats_to_mongo()

no subcategories loaded
subcategories added 0, duplicates not loaded 5


### Extract all unique pages for unique subcategories 

In [9]:
%%time
wiki.wiki_pages()

Unique pages to be added 255 , approx load time (.5 seconds per page) 2.1mins
CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 1.41 s


In [10]:
len(wiki.pages)

255

In [11]:
pages_df=pd.DataFrame(wiki.pages)
pages_df.pull_category.value_counts().head()

Category:Artificial neural networks    122
Category:Classification algorithms      81
Category:Neural network software        20
Category:Deep learning                  18
Category:Decision trees                  9
Name: pull_category, dtype: int64

### For each page pull article extract and category list for  all unique pages and load to page_collection in mongo db
- For development process load to pagetest collection
- pageid assigned as primary key - if page already in mongo it will not be loaded
- keep track of categories a page belongs to as well as the main category it was pulled from


In [12]:
%%time
wiki.load_articles_test()

loading page 1, (1+ε)-approximate nearest neighbor search
loading page 2, Calibration (statistics)
loading page 3, Soft independent modelling of class analogies
loading page 4, Statistical classification
loading page 5, Variable kernel density estimation
loading page 6, AdaBoost
loading page 7, ALOPEX
loading page 8, Alternating decision tree
loading page 9, Analogical modeling
loading page 10, Averaged one-dependence estimators
loading page 11, Artificial neural network
loading page 12, Types of artificial neural networks
loading page 13, Boosting (machine learning)
loading page 14, BrownBoost
loading page 15, C4.5 algorithm
loading page 16, Cascading classifiers
loading page 17, Case-based reasoning
loading page 18, Chi-square automatic interaction detection
loading page 19, Sukhotin's algorithm
loading page 20, Classifier chains
loading page 21, Co-training
loading page 22, CoBoosting
loading page 23, Compositional pattern-producing network
loading page 24, Conceptual clustering
loa

loading page 202, Word embedding
loading page 203, Word2vec
loading page 204, Decision tree
loading page 205, Decision stump
loading page 206, Decision tree model
loading page 207, Fast-and-frugal trees
loading page 208, Gradient boosting
loading page 209, Grafting (decision trees)
loading page 210, Incremental decision tree
loading page 211, Logistic model tree
loading page 212, Pruning (decision trees)
loading page 213, Ensemble learning
loading page 214, Bootstrap aggregating
loading page 215, Gaussian process emulator
loading page 216, LPBoost
loading page 217, Out-of-bag error
loading page 218, AIVA
loading page 219, AlexNet
loading page 220, Apache SINGA
loading page 221, Comparison of deep learning software
loading page 222, Deep Image Prior
loading page 223, Deep Instinct
loading page 224, DeepMind
loading page 225, Keras
loading page 226, Microsoft Cognitive Toolkit
loading page 227, Msg.ai
loading page 228, MXNet
loading page 229, Neural Designer
loading page 230, Numenta
loa

## Check pages loaded to MongoDB

In [13]:
import pymongo
IP='52.42.152.84'
client = pymongo.MongoClient(IP, 27016)
myWiki_dbr=client.myWiki

In [16]:
pagetestload_cllr=myWiki_dbr.pagetestload_collection
cursor=pagetestload_cllr.find()
docs=list(cursor)
docs

[{'_id': ObjectId('5a6a16d7cd7a8900a1f8496a'),
  'loaded': 'Thu Jan 25 17:41:43 2018',
  'master_cat': 'Category:Classification algorithms',
  'pages_added': 255}]

In [14]:
pagetest_cllr=myWiki_dbr.pagetest_collection
cursor=pagetest_cllr.find()
docs=list(cursor)
display(len(docs))
docs[0]

255

{'_id': 32003319,
 'category_list': ['Category:Algorithms and data structures stubs',
  'Category:All stub articles',
  'Category:Approximation algorithms',
  'Category:Classification algorithms',
  'Category:Computer science stubs',
  'Category:Search algorithms'],
 'extract': 'approximate near neighbor search special case near neighbor search problem solution approximate near neighbor search point multiple point within distance query point distance query point true near neighbor reasons approximate near neighbor search include space time cost exact solution high dimensional space see curse dimensionality domain find approximate near neighbor acceptable solution approaches solve approximate near neighbor search include kd tree locality sensitive hashing brute force search reference',
 'ns': 0,
 'pageid': 32003319,
 'pull_category': 'Category:Classification algorithms',
 'title': '(1+ε)-approximate nearest neighbor search'}

In [15]:
docs[1]

{'_id': 740955,
 'category_list': ['Category:Classification algorithms',
  'Category:Regression analysis',
  'Category:Statistical classification'],
 'extract': 'two main use term calibration statistic denote special type statistical inference problem thus calibration mean reverse process regression instead future dependent variable predict know explanatory variable know observation dependent variable use predict correspond explanatory variable procedures statistical classification determine class membership probability assess uncertainty give new observation belong already establish class addition calibration use statistic usual general mean calibration example model calibration conjurer use refer bayesian inference value model parameter give datum set much generally type fit statistical model dawid put forecaster good calibrate example event assign probability percent long run proportion actually occur turn percent regression calibration problem regression use know datum observe rela